In [107]:
example_1 = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

In [108]:
example_2 = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

In [109]:
with open('./data/Day 20/input.txt') as input_file:
    input_text = input_file.read()

In [110]:
def parse_input(lines):
    rules = {}
    for line in lines.splitlines():
        line = line.replace(" ","")
        if "->" in line:
            rule, output = line.split("->")
            if "%" in rule or "&" in rule:
                node = rule[1:].strip()
                nodetype= rule[0]
            else:
                node = rule.strip()
                nodetype= "broadcast"
            rules[node] = {"outputs": output.strip().split(","), "type": nodetype}
    return rules

In [111]:
def flipflop(state, input_signal):
    for key in input_signal:
        if input_signal[key]:
            return None, state
        else:
            if state:
                return 0, 0
            else:
                return 1, 1

def conjunction(state, input_signal):
    for key in input_signal:
        if key not in state:
            raise NotImplementedError
        state[key] = input_signal[key]
    if all(state.values()):
        return 0, state
    else:
        return 1, state

In [112]:
translate = {"%": flipflop,
 "&": conjunction}

In [113]:
def get_conjunctions(actions):
    conjunctions = []
    for key, value in actions.items():
        if value["type"] == "&":
            conjunctions.append(key)
    return conjunctions

def get_flipflops(actions):
    flipflops = []
    for key, value in actions.items():
        if value["type"] == "%":
            flipflops.append(key)
    return flipflops

In [114]:
def init(actions):
    ### Conjunctions
    conjunctions = get_conjunctions(actions)
    conjunction_states = {}
    for key in conjunctions:
        conjunction_states[key] = {}
        for k, value in actions.items():
            if key in value["outputs"]:
                conjunction_states[key][k] = 0
    ### FlipFlops
    flipflops = get_flipflops(actions)
    flipflop_states = {}
    for key in flipflops:
        flipflop_states[key] = 0
    return {**conjunction_states, **flipflop_states}

In [121]:
def propegate_signal(actions, state, signal):
    new_signals = []
    for key, value in signal.items():
        new_signal = {}
        if key not in actions:
            continue
        if actions[key]["type"] == "broadcast":
            for i in actions[key]["outputs"]:
                new_signal[i] = {}
                new_signal[i][key] = list(value.values())[0]
        else:
            if actions[key]["type"] == "%":
                out, state[key] = flipflop(
                    state[key], value
                )    
                for i in actions[key]["outputs"]:
                    if i in new_signal:
                        raise Exception
                    new_signal[i] = {}
                    if out is not None:
                        new_signal[i][key] = out
                    else:
                        del new_signal[i]
            elif actions[key]["type"] == "&":
                out, state[key] = conjunction(
                state[key], value
            )
                for i in actions[key]["outputs"]:
                    if i in new_signal:
                        raise Exception
                    new_signal[i] = {}
                    new_signal[i][key] = out
        new_signals.append(new_signal)
    return new_signals, state

In [122]:
def count_signals(signals, nr_low, nr_high):
    for signal in signals:
        for i, j in signal.items():
            for k, l in j.items():
                if l:
                    nr_high += 1
                else:
                    nr_low += 1
    return nr_low, nr_high

In [123]:
def execute_button_press(actions, state, signals, nr_low, nr_high):
    nr_low, nr_high = count_signals([signals], nr_low, nr_high)
    signals = [signals]
    while signals:
        new_signals = []
        for signal_group in signals:
            signals, state = propegate_signal(actions, state, signal_group)
            new_signals += signals
        signals= new_signals
        nr_low, nr_high = count_signals(signals, nr_low, nr_high)
    return state, signals, nr_low, nr_high

# Part 1

In [124]:
import copy
parsed_example = parse_input(example_1)
example_initial_state = init(parsed_example)
state = copy.deepcopy(example_initial_state)
button_signal = {"broadcaster":{"entry": 0}}
nr_low = 0
nr_high = 0

for nr, button_press in enumerate(range(1000)):
    state, signals, nr_low, nr_high = execute_button_press(parsed_example, state, button_signal, nr_low, nr_high)
    if signals:
        print("ERROR")
nr_low * nr_high

32000000

In [125]:
import copy
parsed_example = parse_input(example_2)
example_initial_state = init(parsed_example)
state = copy.deepcopy(example_initial_state)
button_signal = {"broadcaster":{"entry": 0}}
nr_low = 0
nr_high = 0

for nr, button_press in enumerate(range(1000)):
    state, signals, nr_low, nr_high = execute_button_press(parsed_example, state, button_signal, nr_low, nr_high)
    if signals:
        print("ERROR")
nr_low * nr_high

11687500

In [126]:
import copy
parsed = parse_input(input_text)
initial_state = init(parsed)
state = copy.deepcopy(initial_state)
button_signal = {"broadcaster":{"entry": 0}}
nr_low = 0
nr_high = 0

for nr, button_press in enumerate(range(1000)):
    state, signals, nr_low, nr_high = execute_button_press(parsed, state, button_signal, nr_low, nr_high)
    if signals:
        print("ERROR")
    if state == initial_state:
        print("Loop found at iteration", nr)
        break

# > 492871302
nr_low * nr_high

869395600

# Part 2

In [163]:
def execute_button_press_part2(actions, state, signals, found):
    signals = [signals]
    while signals:
        new_signals = []
        for signal_group in signals:
            signals, state = propegate_signal(actions, state, signal_group)
            new_signals += signals
        signals= new_signals
        for signal in signals:
            if 'ls' in signal:
                for key in signal['ls']:
                    if key in found and signal['ls'][key]:
                        found[key] = True
    return state, signals, found

In [164]:
goal_keys = list(initial_state['ls'].keys())

In [165]:
import copy
parsed = parse_input(input_text)
initial_state = init(parsed)
state = copy.deepcopy(initial_state)
button_signal = {"broadcaster":{"entry": 0}}

found = {key: False for key in goal_keys}
counter_dict = {}
counter = 0
while not all(found.values()):
    counter += 1
    state, signals, found = execute_button_press_part2(parsed, state, button_signal, found)
    for key in found:
        if found[key] and key not in counter_dict:
            counter_dict[key] = counter

In [168]:
import math
math.lcm(*list(counter_dict.values()))

232605773145467